In [1]:
import pandas as pd
import numpy as np
import json, requests
import os
from dotenv import load_dotenv
load_dotenv()
import requests
from pandas.io.json import json_normalize
from SRC.geo_func import *

# Foursquare's Places API

In [2]:
def foursquareAPI(search):
    url = 'https://api.foursquare.com/v2/venues/explore'

    params = dict(
    client_id=os.getenv('CLIENT_ID'),
    client_secret=os.getenv('CLIENT_SECRET'),
    v='20180323', 
    ll="47.62405,-122.33279",
    query=search,
    limit=100
    )
    
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)

    return data

------

## Creating Starbucks Dataframe

In [3]:
starbucks = foursquareAPI("Starbucks")

In [4]:
starbucks_seattle=[]
for i in list(range(len(starbucks["response"]["groups"][0]["items"]))):
        
            starbucks_seattle.append({
        "name": starbucks["response"]["groups"][0]["items"][i]["venue"]["name"],
        "distance": starbucks["response"]["groups"][0]["items"][i]["venue"]["location"]["distance"],
        "latitude": starbucks["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": starbucks["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

In [5]:
df_starbucks = pd.DataFrame(list(starbucks_seattle))

df_starbucks["geopoint"] = df_starbucks.apply(transformToGeoPoint, axis=1)
df_starbucks.head(3)

,name,distance,latitude,longitude,geopoint
0,Starbucks,442,47.624413,-122.338658,"{'type': 'Point', 'coordinates': [-122.3386576..."
1,Starbucks,341,47.622789,-122.336932,"{'type': 'Point', 'coordinates': [-122.3369318..."
2,Starbucks,784,47.619377,-122.324970,"{'type': 'Point', 'coordinates': [-122.32497, ..."


------------

## Creating Airport Dataframe

In [6]:
airports = foursquareAPI("Airport")

In [7]:
airports_seattle=[]
for i in list(range(len(airports["response"]["groups"][0]["items"]))):
        
            airports_seattle.append({
        "name": airports["response"]["groups"][0]["items"][i]["venue"]["name"],
        "distance": airports["response"]["groups"][0]["items"][i]["venue"]["location"]["distance"],
        "latitude": airports["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": airports["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

In [8]:
df_airports = pd.DataFrame(list(airports_seattle))

df_airports["geopoint"] = df_airports.apply(transformToGeoPoint, axis=1)
df_airports.head(3)

,name,distance,latitude,longitude,geopoint
0,Seattle-Tacoma International Airport (SEA) (Se...,20217,47.443589,-122.302508,"{'type': 'Point', 'coordinates': [-122.3025083..."
1,Kenmore Air,723,47.628535,-122.339765,"{'type': 'Point', 'coordinates': [-122.3397651..."
2,Kenmore Air Flight M5310,729,47.628541,-122.339864,"{'type': 'Point', 'coordinates': [-122.339864,..."


-----------

## Creating Schools Dataframe

In [47]:
children = foursquareAPI("School")

In [48]:
children_seattle=[]
for i in list(range(len(children["response"]["groups"][0]["items"]))):
        
            children_seattle.append({
        "name": children["response"]["groups"][0]["items"][i]["venue"]["name"],
        "distance": children["response"]["groups"][0]["items"][i]["venue"]["location"]["distance"],
        "latitude": children["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": children["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

In [49]:
df_children = pd.DataFrame(list(children_seattle))

df_children["geopoint"] = df_children.apply(transformToGeoPoint, axis=1)
df_children.head(3)

,name,distance,latitude,longitude,geopoint
0,General Assembly,212,47.623304,-122.330178,"{'type': 'Point', 'coordinates': [-122.3301780..."
1,Events by Heather & Ryan,214,47.623428,-122.330084,"{'type': 'Point', 'coordinates': [-122.3300838..."
2,Code Fellows,250,47.623399,-122.335989,"{'type': 'Point', 'coordinates': [-122.3359893..."


-------------

## Creating Vegan Dataframe

In [9]:
vegan = foursquareAPI("Vegan")

In [10]:
vegan_seattle=[]
for i in list(range(len(vegan["response"]["groups"][0]["items"]))):
        
            vegan_seattle.append({
        "name": vegan["response"]["groups"][0]["items"][i]["venue"]["name"],
        "distance": vegan["response"]["groups"][0]["items"][i]["venue"]["location"]["distance"],
        "latitude": vegan["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": vegan["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

In [11]:
df_vegan = pd.DataFrame(list(vegan_seattle))

df_vegan["geopoint"] = df_vegan.apply(transformToGeoPoint, axis=1)
df_vegan.head(3)

,name,distance,latitude,longitude,geopoint
0,Veggie Grill,328,47.623078,-122.336920,"{'type': 'Point', 'coordinates': [-122.3369198..."
1,Kati Vegan Thai,357,47.620864,-122.333340,"{'type': 'Point', 'coordinates': [-122.3333401..."
2,Wedgwood II Vegetarian Thai,909,47.622658,-122.320841,"{'type': 'Point', 'coordinates': [-122.3208408..."


----------

----------

--------

# MongoDB Compass $near Queries

In [30]:
from pymongo import MongoClient
dbName = "tech_startups"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()
cursor = db.list_collections()
from pymongo import ASCENDING, DESCENDING

--------------

## Starbucks nearby

In [40]:
offi_starbucks=[]

for i in range(len(df_starbucks)):
    uno = list(db.offices.find(geoQueryNear(df_starbucks.iloc[i].geopoint, radius=1000),{"_id":0}))
    offi_starbucks.append(uno)  

In [45]:
offi_starbucks[0]

[{'name': 'Shelfari',
  'category_code': 'web',
  'total_money_raised': '$1M',
  'description': None,
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.620716,
  'longitude': -122.347533,
  'geopoint': {'type': 'Point', 'coordinates': [-122.347533, 47.620716]}}]

## Flattening it so it can be put into a df

In [42]:
offi_starbucks2 = [x for e in offi_starbucks for x in e]

In [58]:
df_offi_starbucks = pd.DataFrame(offi_starbucks2)
len(df_offi_starbucks)

18

In [35]:
df_offi_starbucks.to_json("output/df_offi_starbucks.json",orient="records")

# Terminal=>  mongoimport -d tech_startups -c nearby_starbucks --jsonArray df_offi_starbucks.json

---------

## Starbucks and Airports nearby

In [36]:
offi_airports=[]

for i in range(len(df_airports)):
    dos = list(db.nearby_starbucks.find(geoQueryNear(df_airports.iloc[i].geopoint, radius=20000),{"_id":0}))
    offi_airports.append(dos)             

In [37]:
offi_airports[0]

[{'name': 'Napera Networks',
  'category_code': 'network_hosting',
  'total_money_raised': '$6M',
  'description': 'Headquarters',
  'address1': '4957 Lakemont Blvd SE',
  'address2': 'Suite C4-300',
  'zip_code': '98006',
  'city': 'Bellevue',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.578174,
  'longitude': -122.212731,
  'geopoint': {'type': 'Point', 'coordinates': [-122.212731, 47.578174]}},
 {'name': 'Napera Networks',
  'category_code': 'network_hosting',
  'total_money_raised': '$6M',
  'description': 'Headquarters',
  'address1': '4957 Lakemont Blvd SE',
  'address2': 'Suite C4-300',
  'zip_code': '98006',
  'city': 'Bellevue',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.578174,
  'longitude': -122.212731,
  'geopoint': {'type': 'Point', 'coordinates': [-122.212731, 47.578174]}},
 {'name': 'Napera Networks',
  'category_code': 'network_hosting',
  'total_money_raised': '$6M',
  'description': 'Headquarters',
  'address1': '4957 Lakemont 

## Flattening it so it can be put into a df

In [38]:
offi_airports2 = [x for e in offi_airports for x in e]

In [57]:
df_offi_airports= pd.DataFrame(offi_airports2)
len(df_offi_airports)

1518

In [46]:
df_offi_airports.to_json("output/df_offi_airports.json",orient="records")

# Terminal=>  mongoimport -d tech_startups -c nearby_airports --jsonArray df_offi_airports.json

-------------

## Starbucks, Airports and Schools Nearby

In [51]:
offi_schools=[]

for i in range(len(df_children)):
    tres = list(db.nearby_airports.find(geoQueryNear(df_children.iloc[i].geopoint, radius=5000),{"_id":0}))
    offi_schools.append(tres)             

In [52]:
offi_schools[0]

[{'name': 'Shelfari',
  'category_code': 'web',
  'total_money_raised': '$1M',
  'description': None,
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.620716,
  'longitude': -122.347533,
  'geopoint': {'type': 'Point', 'coordinates': [-122.347533, 47.620716]}},
 {'name': 'Shelfari',
  'category_code': 'web',
  'total_money_raised': '$1M',
  'description': None,
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.620716,
  'longitude': -122.347533,
  'geopoint': {'type': 'Point', 'coordinates': [-122.347533, 47.620716]}},
 {'name': 'Shelfari',
  'category_code': 'web',
  'total_money_raised': '$1M',
  'description': None,
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.620716,
  'longitude': -122.347533,
  'geopoint': {'typ

## Flattening it so it can be put into a df

In [53]:
offi_schools2 = [x for e in offi_schools for x in e]

In [63]:
df_offi_schools= pd.DataFrame(offi_schools2)
df_offi_schools

,name,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geopoint
0,Shelfari,web,$1M,None,,,,Seattle,WA,USA,47.620716,-122.347533,"{'type': 'Point', 'coordinates': [-122.347533,..."
1,Shelfari,web,$1M,None,,,,Seattle,WA,USA,47.620716,-122.347533,"{'type': 'Point', 'coordinates': [-122.347533,..."
2,Shelfari,web,$1M,None,,,,Seattle,WA,USA,47.620716,-122.347533,"{'type': 'Point', 'coordinates': [-122.347533,..."
3,Shelfari,web,$1M,None,,,,Seattle,WA,USA,47.620716,-122.347533,"{'type': 'Point', 'coordinates': [-122.347533,..."
4,Shelfari,web,$1M,None,,,,Seattle,WA,USA,47.620716,-122.347533,"{'type': 'Point', 'coordinates': [-122.347533,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81595,Kashless,ecommerce,$9M,World Headquarters,106 West Roy St.,,98119,Seattle,WA,USA,47.625740,-122.358162,"{'type': 'Point', 'coordinates': [-122.3581623..."
81596,Kashless,ecommerce,$9M,World Headquarters,106 West Roy St.,,98119,Seattle,WA,USA,47.625740,-122.358162,"{'type': 'Point', 'coordinates': [-122.3581623..."
81597,Kashless,ecommerce,$9M,World Headquarters,106 West Roy St.,,98119,Seattle,WA,USA,47.625740,-122.358162,"{'type': 'Point', 'coordinates': [-122.3581623..."
81598,Kashless,ecommerce,$9M,World Headquarters,106 West Roy St.,,98119,Seattle,WA,USA,47.625740,-122.358162,"{'type': 'Point', 'coordinates': [-122.3581623..."


In [ ]:
df_offi_schools.to_json("output/df_offi_schools.json",orient="records")

# Terminal=>  mongoimport -d tech_startups -c nearby_airports --jsonArray df_offi_airports.json

## Starbucks, Airports, Schools and Vegan Nearby

In [65]:
offi_vegan=[]

for i in range(len(df_vegan)):
    tres = list(db.nearby_airports.find(geoQueryNear(df_vegan.iloc[i].geopoint, radius=5000),{"_id":0}))
    offi_vegan.append(tres)             

In [66]:
offi_vegan[0]

[{'name': 'Shelfari',
  'category_code': 'web',
  'total_money_raised': '$1M',
  'description': None,
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.620716,
  'longitude': -122.347533,
  'geopoint': {'type': 'Point', 'coordinates': [-122.347533, 47.620716]}},
 {'name': 'Shelfari',
  'category_code': 'web',
  'total_money_raised': '$1M',
  'description': None,
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.620716,
  'longitude': -122.347533,
  'geopoint': {'type': 'Point', 'coordinates': [-122.347533, 47.620716]}},
 {'name': 'Shelfari',
  'category_code': 'web',
  'total_money_raised': '$1M',
  'description': None,
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.620716,
  'longitude': -122.347533,
  'geopoint': {'typ

## Flattening it so it can be put into a df

In [67]:
offi_vegan2 = [x for e in offi_vegan for x in e]

In [68]:
df_offi_vegan= pd.DataFrame(offi_vegan2)
len(df_offi_vegan)

22950

In [ ]:
df_offi_vegan.to_json("output/df_offi_vegan.json",orient="records")

# Terminal=>  mongoimport -d tech_startups -c nearby_airports --jsonArray df_offi_airports.json